# 자동 미분

In [13]:
import torch

`autograd`는 PyTorch에서 핵심적인 기능을 담당하는 하부 패키지이다. 

autograd는 텐서의 연산에 대해 자동으로 미분값을 구해주는 기능을 한다.

- 텐서 자료를 생성할 때, `requires_grad`인수를 `True`로 설정하거나 
- `.requires_grad_(True)`를 실행하면

그 텐서에 행해지는 모든 연산에 대한 미분값을 계산한다. 

계산을 멈추고 싶으면 `.detach()`함수나 with을 이용해 `torch.no_grad()`를  이용하면 된다.

In [14]:
x = torch.rand(2, 2)

In [15]:
x

tensor([[0.8200, 0.6864],
        [0.6492, 0.6429]])

In [16]:
x = torch.rand(2, 2, requires_grad=True)

In [17]:
x

tensor([[0.6995, 0.6948],
        [0.6637, 0.7060]], requires_grad=True)

In [18]:
# 미분값을 트래킹 하겠다고 선언

다음으로 이 x에 연산을 수행한다. 다음 코드의 y는 연산의 결과이므로 미분 함수를 가진다. `grad_fn`속성을 출력해 미분 함수를 확인 할 수 있다. 

In [19]:
y  = torch.sum(x * 3)

In [20]:
print(y, y.grad_fn)
            # 속성값이 있는지가 중요

tensor(8.2922, grad_fn=<SumBackward0>) <SumBackward0 object at 0x117e15a90>


In [21]:
# tape.gradient(y, x)
y.backward(retain_graph=True)

In [22]:
x.grad

tensor([[3., 3.],
        [3., 3.]])

`y.backward()` 함수를 실행하면 x의 미분값이 자동으로 갱신된다. x의 `grad`속성을 확인하면 미분값이 들어 있는 것을 확인 할 수 있다. y를 구하기 위한 x의 연산을 수식으로 쓰면 다음과 같다. 

$$
y = \displaystyle\sum_{i=1}^4 3 \times x_i
$$

이를 $x_i$에 대해 미분 하면 미분 함수는 다음과 같다. 

$$
\dfrac{\partial y}{\partial x_i} = 3
$$

실제 미분값과 같은지 확인해보자.

`backward()`함수는 자동으로 미분값을 계산해 `requires_grad`인수가 True로 설정된 변수의 `grad`속성의 값을 갱신한다. 

`retain_graph` 미분을 연산하기 위해서 사용했던 임시 그래프를 유지 할 것인가를 설정하는 것이다. 
기본값은 False로 설정되어 있지만 동일한 연산에 대해 여러번 미분을 계산하기 위해서는 True로 설정되어 있어야한다.(`tf.GradientTape`에서 `persistent`와 같음)

`torch.autograd.grad()`함수를 사용해 `tf.GradientTape`처럼 사용할 수도 있다. 

In [23]:
torch.autograd.grad(y, x)       # 미분 값이 나온다.

(tensor([[3., 3.],
         [3., 3.]]),)

상황에 따라 특정 연산에는 미분값을 계산하고 싶지 않은 경우에는 

 - `.detach()`함수
 - `with torch.no_grad()`

In [25]:
print(x.grad)
x_d = x.detach()    # 미분값을 트래킹 하지 않는 것. // 학습할 때, 로그를 남길때 많이 쓴다.
torch.sigmoid(x_d)

tensor([[3., 3.],
        [3., 3.]])


tensor([[0.6681, 0.6670],
        [0.6601, 0.6695]])

In [26]:
x.grad

tensor([[3., 3.],
        [3., 3.]])

In [28]:
print(x_d.grad)

None


In [30]:
# torch.no_grad()
print(x.grad)

with torch.no_grad():
    x_d2 = torch.sigmoid(x)
    print(x_d2.grad)

tensor([[3., 3.],
        [3., 3.]])
None
